
# Llama fine-tuning with Ludwig


Voir https://ludwig.ai/latest/ et https://colab.research.google.com/drive/1Ly01S--kUwkKQalE-75skalp-ftwl0fE?usp=sharing

Il faut obtenir les droits sur Llama : https://ai.meta.com/resources/models-and-libraries/llama-downloads/

Puis les demander sur HF : https://huggingface.co/meta-llama/Llama-2-7b-hf


In [1]:
!pip install "ludwig[llm]" --quiet

In [2]:
from decouple import Config, RepositoryEnv
import os
DOTENV_FILE = '../.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
os.environ["HUGGING_FACE_HUB_TOKEN"] = env_config('HUGGING_FACE_HUB_TOKEN')

In [3]:
import yaml

config_str = """
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
  bits: 4

adapter:
  type: lora

prompt:
  template: |
    ### Instruction:
    {instruction}

    ### Input:
    {input}

    ### Response:

input_features:
  - name: prompt
    type: text

output_features:
  - name: output
    type: text

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1
"""

config = yaml.safe_load(config_str)

In [4]:
import logging
from ludwig.api import LudwigModel


model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
print(results)

/media/data-nvme/dev/src/llm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Created a temporary directory at /tmp/tmpgkbat4b2
Writing /tmp/tmpgkbat4b2/_remote_module_non_scriptable.py


ConfigValidationError: Specified base model `meta-llama/Llama-2-7b-hf` is not a valid pretrained CausalLM listed on huggingface. Please see: https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads